In [1]:
import os
from typing import List, Dict

from pymilvus import MilvusClient, DataType
from sentence_transformers import SentenceTransformer
from pypdf import PdfReader
from dotenv import load_dotenv
load_dotenv()

# ========= CONFIGURATION =========

# Set these as environment variables OR hard-code (not recommended)
MILVUS_HOST = os.getenv("MILVUS_HOST")        # gRPC host from watsonx.data Milvus service
MILVUS_PORT = os.getenv("MILVUS_PORT", "443") # gRPC port from service
MILVUS_API_KEY = os.getenv("MILVUS_API_KEY")  # IBM Cloud API key

# # Paths to your PDFs
# PUBLIC_PDF_PATH = "./data/offerings_public.pdf"
# MANAGERS_PDF_PATH = "./data/offerings_managers_only.pdf"

# Collection names
PUBLIC_COLLECTION = "offerings_public"
MANAGERS_COLLECTION = "offerings_managers_only"

# Embedding model (384 dimensions)
EMBEDDING_MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
EMBEDDING_DIM = 384  # fixed for this model

# print(MILVUS_HOST)
# print(MILVUS_PORT)
# print(MILVUS_API_KEY)

/Users/michal/anaconda3/envs/milvus/lib/python3.11/site-packages/pymilvus/client/__init__.py:6: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution
/Users/michal/anaconda3/envs/milvus/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def connect_milvus() -> MilvusClient:
    if not (MILVUS_HOST and MILVUS_PORT and MILVUS_API_KEY):
        raise RuntimeError("Set MILVUS_HOST, MILVUS_PORT and MILVUS_API_KEY first.")
	

    milvus_uri = f"https://ibmlhapikey_michal.kordyzon@pl.ibm.com:{MILVUS_API_KEY}@{MILVUS_HOST}:{MILVUS_PORT}"

    client = MilvusClient(
        uri=milvus_uri,
        secure=True,
    )
    return client


In [3]:
from typing import List, Dict
from pprint import pprint

def semantic_search_public(
    client,
    model,
    query: str,
    top_k: int = 5,
) -> List[Dict]:
    """
    Semantic search over the 'offerings_public' collection.
    Uses SentenceTransformer for embeddings and MilvusClient.search for ANN.
    """
    # 1. Ensure collection is loaded
    client.load_collection(PUBLIC_COLLECTION)

    # 2. Embed the query
    query_vec = model.encode([query], convert_to_numpy=True)[0].tolist()

    # 3. Search parameters
    search_params = {
        "metric_type": "COSINE",  # must match index metric
        "params": {}              # FLAT index → no extra params
    }

    # 4. Search in Milvus
    results = client.search(
        collection_name=PUBLIC_COLLECTION,
        data=[query_vec],
        anns_field="embedding",
        limit=top_k,
        output_fields=["offering_id", "text"],
        search_params=search_params,
    )

    hits = results[0]

    formatted = []
    for hit in hits:
        entity = hit.get("entity", {})
        formatted.append(
            {
                "id": hit["id"],
                "score": hit["distance"],              # COSINE distance; lower = closer
                "offering_id": entity.get("offering_id"),
                "text": entity.get("text", ""),
            }
        )

    return formatted


In [4]:
client = connect_milvus()
print("Connected to IBM Milvus.")

print("Loading embedding model (if not already loaded)...")
model = SentenceTransformer(EMBEDDING_MODEL_NAME)

query = "instant travel insurance for flight delays"
results = semantic_search_public(
    client, 
    model, 
    query=query, 
    top_k=5)

def semantic_search_public(
    client,
    model,
    query: str,
    top_k: int = 5,
) -> List[Dict]:
    """
    Semantic search over the 'offerings_public' collection.
    Uses SentenceTransformer for embeddings and MilvusClient.search for ANN.
    """
    # 1. Ensure collection is loaded
    client.load_collection(PUBLIC_COLLECTION)

    # 2. Embed the query
    query_vec = model.encode([query], convert_to_numpy=True)[0].tolist()

    # 3. Search parameters
    search_params = {
        "metric_type": "COSINE",  # must match index metric
        "params": {}              # FLAT index → no extra params
    }

    # 4. Search in Milvus
    results = client.search(
        collection_name=PUBLIC_COLLECTION,
        data=[query_vec],
        anns_field="embedding",
        limit=top_k,
        output_fields=["offering_id", "text"],
        search_params=search_params,
    )

    hits = results[0]

    formatted = []
    for hit in hits:
        entity = hit.get("entity", {})
        formatted.append(
            {
                "id": hit["id"],
                "score": hit["distance"],              # COSINE distance; lower = closer
                "offering_id": entity.get("offering_id"),
                "text": entity.get("text", ""),
            }
        )

    return formatted


print(f"\nTop results for query: {query!r}")
for i, r in enumerate(results, 1):
    print("\n---- Result", i, "----")
    print("Score      :", r["score"])
    print("ID         :", r["id"])
    print("Offering ID:", r["offering_id"])
    print("Text       :", r["text"][:400], "...")


Connected to IBM Milvus.
Loading embedding model (if not already loaded)...

Top results for query: 'instant travel insurance for flight delays'

---- Result 1 ----
Score      : 0.5181092619895935
ID         : 462349868925426229
Offering ID: offering_xyz
Text       : FintechNova – Token-Based Insurance Portfolio (Public Overview) FintechNova is introducing a next-generation suite of token-based insurance products designed for digital-first customers who value transparency, automation, and rapid claim settlement. All products are built on our proprietary NovaChain technology, which enables secure token issuance, premium automation, and instant micro-payouts usi ...

---- Result 2 ----
Score      : 0.43706774711608887
ID         : 462349868925426230
Offering ID: offering_xyz
Text       : Safe driving → lower premiums and bonus tokens. • Immediate damage assessment using AI incident detection. • Smart roadside assistance: Request support directly through your token wallet. Target Users: U